In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

from scipy import optimize, stats

In [ ]:
plt.rcParams.update({'font.size': 16})
# mpl.rcParams['lines.linewidth'] = 4.

dosave = True
savedir = "plots_1_29_24/signalbox/"

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/mc-F/"
filetype = "F-CohLike_%s_evt_spectrum.df"
# filetype = "F2-Higgs_M260_%s_evt_spectrum.df"


masses = [220, 240, 260, 280, 300, 320, 340]

filetypes = ["F2-Higgs_M%i_%s_evt_blah_spectrum.df" % (M, "%s") for M in masses]
ftype_names = ["$M_S: $ %i MeV" % M for M in masses]

In [ ]:
nominal = "nom"

variations = [
    "ind0nom",
    "noiselhi",
    "ind1bin0",
    "ind1bin14",
    "sce2x",
    "ind0glo",
    "ind0glo",
    "nom_gainlo",
    "nom_gainhi",
    "nom_callo",
    "nom_calhi",
    "nom_mcslo",
    "nom_mcshi"
]

vnames = [
    "Front Ind. W.C.",
    "Noise 1.1x",
    "Middle Ind. Opaque",
    "Middle Ind. Transparent",
    "SCE 2x",
    "Ind. Gain 0.85x",
    "Ind. Gain 1.15x",
    
    "Cal. Gain 1.01x",
    "Cal. Gain 0.99x",
    "Cal. dE/dx Low",
    "Cal. dE/dx High",
    "MCS Low",
    "MCS High"
]

In [ ]:
cvdfs = [pd.read_hdf(filedir + ft % nominal) for ft in filetypes]

vardfs = [[pd.read_hdf(filedir + ft % v) for v in variations] for ft in filetypes]

In [ ]:
def fit_peak(X, A, mu, sigma, m, B):
    Xp = -(X-mu)/sigma
    return A*stats.crystalball.pdf(Xp, m, B)

In [ ]:
bins = np.linspace(0.2, 0.4, 31)
centers = (bins[:-1] + bins[1:]) / 2

cv_sigmas = []

cv_sigma_errs = []
for cvdf in cvdfs:
    N,_= np.histogram(cvdf.mass, bins=bins)

    popt,perr = optimize.curve_fit(fit_peak, centers, N, 
                                sigma=np.maximum(np.sqrt(N),1), p0=[np.sum(N), 0.22, 0.01, 1, 5],
                               maxfev=int(1e6))
    sigma = popt[2]
    
    err = np.sqrt(np.diag(perr))[2]
    
    cv_sigmas.append(sigma*1e3)
    cv_sigma_errs.append(err*1e3)
    
cv_sigmas = np.array(cv_sigmas)
cv_sigma_errs = np.array(cv_sigma_errs)

In [ ]:
bins = np.linspace(0.2, 0.4, 31)
centers = (bins[:-1] + bins[1:]) / 2

var_sigmas = []
for ift, vdfs in enumerate(vardfs):
    var_sigmas.append([])
    for ivar, vdf in enumerate(vdfs):
        N,_= np.histogram(vdf.mass, bins=bins)

        popt,_ = optimize.curve_fit(fit_peak, centers, N, 
                                    sigma=np.maximum(np.sqrt(N),1), p0=[np.sum(N), 0.22, 0.01, 1, 5],
                                   maxfev=int(1e6))
        sigma = popt[2]

        var_sigmas[-1].append(sigma*1e3)
var_sigmas = np.array(var_sigmas)

In [ ]:
var_sigmas.shape

In [ ]:
for iv in range(var_sigmas.shape[1]):
    plt.plot(masses, 100*var_sigmas[:, iv] / masses, linestyle="none", marker=".", markersize=10, 
             label=vnames[iv])
    
plt.errorbar(masses, 100*cv_sigmas / masses, yerr=cv_sigma_errs*100/masses, 
             linestyle="none", marker=".", markersize=10, color="black", label="Nominal")

plt.ylim([0.75, 5.5])

plt.legend(ncol=2, loc='upper center', bbox_to_anchor=(0.5, 1.7))
plt.xlabel("Scalar Mass [MeV]")
plt.ylabel("Mass Peak Resolution [%]")

if dosave:
    plt.savefig(savedir + "mass_peak_resolution_systvar.pdf", bbox_inches="tight")
    plt.savefig(savedir + "mass_peak_resolution_systvar.svg", bbox_inches="tight")